In [1]:
import csv
import json
import numpy as np
import colorspacious

In [2]:
ALL_NUM_COLORS = [6, 8, 10]

## Generate LaTeX markup for results table

In [3]:
with open("../aesthetic-models/top-cycles.json") as infile:
    top_cycles = json.load(infile)
top_cycles = {int(i): top_cycles[i] for i in top_cycles}

In [4]:
npz = np.load("../color-name-model/colornamemodel.npz")
COLOR_NAMES = list(npz["names"])
BCT_IDX = npz["bct_idxs"]
COLOR_NAME_IDX = npz["name_idxs"]

In [5]:
# Calculate min delta E for each position in each cycle
min_dists = {}
for nc in ALL_NUM_COLORS:
    rgb = (
        np.array(
            [(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in top_cycles[nc]]
        )
        / 255
    )
    min_dists[nc] = [100]
    min_dist = 100
    for i in range(1, nc):
        for severity in range(1, 101):
            deut = colorspacious.cspace_convert(
                rgb[: i + 1],
                {
                    "name": "sRGB1+CVD",
                    "cvd_type": "deuteranomaly",
                    "severity": severity,
                },
                "sRGB1",
            )
            prot = colorspacious.cspace_convert(
                rgb[: i + 1],
                {"name": "sRGB1+CVD", "cvd_type": "protanomaly", "severity": severity},
                "sRGB1",
            )
            trit = colorspacious.cspace_convert(
                rgb[: i + 1],
                {"name": "sRGB1+CVD", "cvd_type": "tritanomaly", "severity": severity},
                "sRGB1",
            )
            for j in range(i):
                min_dist = min(min_dist, colorspacious.deltaE(rgb[i], rgb[j]))
                min_dist = min(min_dist, colorspacious.deltaE(deut[i], deut[j]))
                min_dist = min(min_dist, colorspacious.deltaE(prot[i], prot[j]))
                min_dist = min(min_dist, colorspacious.deltaE(trit[i], trit[j]))
        min_dists[nc].append(min_dist)

In [6]:
table_output = r"""\begin{tabular}{@{}rrrrrrrrrrrrrrr@{}}
\toprule
\multicolumn{5}{c}{Six Colors} & \multicolumn{5}{c}{Eight Colors} & \multicolumn{5}{c}{Ten Colors} \\ \cmidrule(r){1-5}\cmidrule(rl){6-10}\cmidrule(l){11-15}
& \multicolumn{1}{c}{R} & \multicolumn{1}{c}{G} & \multicolumn{1}{c}{B} & $\min\Delta E_\text{cvd}$ & & \multicolumn{1}{c}{R} & \multicolumn{1}{c}{G} & \multicolumn{1}{c}{B} & $\min\Delta E_\text{cvd}$ & & \multicolumn{1}{c}{R} & \multicolumn{1}{c}{G} & \multicolumn{1}{c}{B} & $\min\Delta E_\text{cvd}$ \\ \midrule
"""
for i in range(10):
    for nc in ALL_NUM_COLORS:
        if i < nc:
            color = top_cycles[nc][i]
            r = int(color[0:2], 16)
            g = int(color[2:4], 16)
            b = int(color[4:], 16)
            name = COLOR_NAMES[COLOR_NAME_IDX[r + g * 256 + b * 256 ** 2]]
            table_output += (
                name + r" \textcolor[HTML]{" + color + r"}{$\blacksquare$} & "
            )
            table_output += f"{r} & {g} & {b} & {min_dists[nc][i]:.1f} & "
        else:
            table_output += "& & & & & "
    table_output = table_output[:-2]
    table_output += "\\\\\n"
table_output += r"""\bottomrule
\end{tabular}"""
print(table_output)

\begin{tabular}{@{}rrrrrrrrrrrrrrr@{}}
\toprule
\multicolumn{5}{c}{Six Colors} & \multicolumn{5}{c}{Eight Colors} & \multicolumn{5}{c}{Ten Colors} \\ \cmidrule(r){1-5}\cmidrule(rl){6-10}\cmidrule(l){11-15}
& \multicolumn{1}{c}{R} & \multicolumn{1}{c}{G} & \multicolumn{1}{c}{B} & $\min\Delta E_\text{cvd}$ & & \multicolumn{1}{c}{R} & \multicolumn{1}{c}{G} & \multicolumn{1}{c}{B} & $\min\Delta E_\text{cvd}$ & & \multicolumn{1}{c}{R} & \multicolumn{1}{c}{G} & \multicolumn{1}{c}{B} & $\min\Delta E_\text{cvd}$ \\ \midrule
blue \textcolor[HTML]{5790fc}{$\blacksquare$} & 87 & 144 & 252 & 100.0 & blue \textcolor[HTML]{1845fb}{$\blacksquare$} & 24 & 69 & 251 & 100.0 & blue \textcolor[HTML]{3f90da}{$\blacksquare$} & 63 & 144 & 218 & 100.0 \\
orange \textcolor[HTML]{f89c20}{$\blacksquare$} & 248 & 156 & 32 & 57.1 & orange \textcolor[HTML]{ff5e02}{$\blacksquare$} & 255 & 94 & 2 & 66.9 & orange \textcolor[HTML]{ffa90e}{$\blacksquare$} & 255 & 169 & 14 & 56.8 \\
red \textcolor[HTML]{e42536}{$\blacksq

## Generate LaTeX markup for sequential-search cycles table

In [7]:
with open("../set-generation/maxdistinct_nc11_cvd100_minj0_maxj100.txt") as csv_file:
    csv_file.readline()
    csv_file.readline()
    csv_file.readline()
    csv_file.readline()
    csv_reader = csv.reader(csv_file, delimiter=" ")
    seq_cycle_a = [["ffffff", 100.0]] + [[row[0], float(row[1])] for row in csv_reader]

with open("../set-generation/maxdistinct_nc11_cvd100_minj40_maxj90.txt") as csv_file:
    csv_file.readline()
    csv_file.readline()
    csv_file.readline()
    csv_file.readline()
    csv_reader = csv.reader(csv_file, delimiter=" ")
    seq_cycle_b = [["ffffff", 100.0]] + [[row[0], float(row[1])] for row in csv_reader]

In [8]:
table_output = r"""\begin{tabular}{@{}crrrrcrrrr@{}}
\toprule
\multicolumn{5}{c}{$J' \in [0, 100]$} & \multicolumn{5}{c}{$J' \in [40, 90]$} \\ \cmidrule(r){1-5}\cmidrule(l){6-10}
& \multicolumn{1}{c}{R} & \multicolumn{1}{c}{G} & \multicolumn{1}{c}{B} & $\Delta E_\text{cvd}$ & & \multicolumn{1}{c}{R} & \multicolumn{1}{c}{G} & \multicolumn{1}{c}{B} & $\Delta E_\text{cvd}$ \\ \midrule
"""
for i in range(1, 11):
    color = seq_cycle_a[i][0]
    r = int(color[0:2], 16)
    g = int(color[2:4], 16)
    b = int(color[4:], 16)
    table_output += r"\textcolor[HTML]{" + color + r"}{$\blacksquare$} & "
    table_output += f"{r} & {g} & {b} & {seq_cycle_a[i][1]:.1f} & "
    color = seq_cycle_b[i][0]
    r = int(color[0:2], 16)
    g = int(color[2:4], 16)
    b = int(color[4:], 16)
    table_output += r"\textcolor[HTML]{" + color + r"}{$\blacksquare$} & "
    table_output += f"{r} & {g} & {b} & {seq_cycle_b[i][1]:.1f} " + "\\\\\n"
table_output += r"""\bottomrule
\end{tabular}"""
print(table_output)

\begin{tabular}{@{}crrrrcrrrr@{}}
\toprule
\multicolumn{5}{c}{$J' \in [0, 100]$} & \multicolumn{5}{c}{$J' \in [40, 90]$} \\ \cmidrule(r){1-5}\cmidrule(l){6-10}
& \multicolumn{1}{c}{R} & \multicolumn{1}{c}{G} & \multicolumn{1}{c}{B} & $\Delta E_\text{cvd}$ & & \multicolumn{1}{c}{R} & \multicolumn{1}{c}{G} & \multicolumn{1}{c}{B} & $\Delta E_\text{cvd}$ \\ \midrule
\textcolor[HTML]{000000}{$\blacksquare$} & 0 & 0 & 0 & 100.0 & \textcolor[HTML]{0045fe}{$\blacksquare$} & 0 & 69 & 254 & 67.8 \\
\textcolor[HTML]{2965ff}{$\blacksquare$} & 41 & 101 & 255 & 59.4 & \textcolor[HTML]{9c3a00}{$\blacksquare$} & 156 & 58 & 0 & 61.8 \\
\textcolor[HTML]{a36300}{$\blacksquare$} & 163 & 99 & 0 & 54.0 & \textcolor[HTML]{908e9e}{$\blacksquare$} & 144 & 142 & 158 & 37.5 \\
\textcolor[HTML]{484854}{$\blacksquare$} & 72 & 72 & 84 & 33.1 & \textcolor[HTML]{ffa100}{$\blacksquare$} & 255 & 161 & 0 & 36.1 \\
\textcolor[HTML]{01f700}{$\blacksquare$} & 1 & 247 & 0 & 32.8 & \textcolor[HTML]{6c4b7d}{$\blacksquare$} &

## Teaser figure with visualization of top cycles

In [9]:
svg = """<svg width="7in" height="2.25in" version="1.1" viewBox="0 0 7 2.25" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
<defs>
<path id="a" d="m0 0c0.4-0.13 0.6-0.13 1 0 0.4 0.13 0.6 0.13 1 0" fill="none" stroke-linecap="round" stroke-width=".02"/>
</defs>
<use transform="translate(.019088 .1169)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(.019088 .47974)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(.019088 .84259)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(.019088 1.2054)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(.019088 1.5683)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(.019088 1.9311)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(2.5 .1169)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(2.5 .37607)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(2.5 .63525)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(2.5 .89443)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(2.5 1.1536)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(2.5 1.4128)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(2.5 1.672)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(2.5 1.9311)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(4.9811 .1169)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(4.9811 .31848)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(4.9811 .52006)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(4.9811 .72164)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(4.9811 .92323)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(4.9811 1.1248)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(4.9811 1.3264)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(4.9811 1.528)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(4.9811 1.7296)" xlink:href="#a" stroke="#{}"/>
<use transform="translate(4.9811 1.9311)" xlink:href="#a" stroke="#{}"/>
<rect x=".01" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x=".39164" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x=".77327" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="1.1549" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="1.5365" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="1.9182" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="2.4959" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="2.7685" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="3.0411" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="3.3137" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="3.5863" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="3.8589" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="4.1315" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="4.4041" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="4.982" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="5.194" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="5.406" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="5.6181" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="5.8301" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="6.0421" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="6.2541" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="6.4661" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="6.6782" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
<rect x="6.8902" y="2.14" width=".1" height=".1" rx=".01" ry=".01" fill="#{}"/>
</svg>"""
print(svg.format(*((top_cycles[6] + top_cycles[8] + top_cycles[10]) * 2)))

<svg width="7in" height="2.25in" version="1.1" viewBox="0 0 7 2.25" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
<defs>
<path id="a" d="m0 0c0.4-0.13 0.6-0.13 1 0 0.4 0.13 0.6 0.13 1 0" fill="none" stroke-linecap="round" stroke-width=".02"/>
</defs>
<use transform="translate(.019088 .1169)" xlink:href="#a" stroke="#5790fc"/>
<use transform="translate(.019088 .47974)" xlink:href="#a" stroke="#f89c20"/>
<use transform="translate(.019088 .84259)" xlink:href="#a" stroke="#e42536"/>
<use transform="translate(.019088 1.2054)" xlink:href="#a" stroke="#964a8b"/>
<use transform="translate(.019088 1.5683)" xlink:href="#a" stroke="#9c9ca1"/>
<use transform="translate(.019088 1.9311)" xlink:href="#a" stroke="#7a21dd"/>
<use transform="translate(2.5 .1169)" xlink:href="#a" stroke="#1845fb"/>
<use transform="translate(2.5 .37607)" xlink:href="#a" stroke="#ff5e02"/>
<use transform="translate(2.5 .63525)" xlink:href="#a" stroke="#c91f16"/>
<use transform="translate(2.

## Generate LaTeX markup for best / worst color sets

In [10]:
COLOR_FILE = {
    6: "../set-generation/colors_mcd20.0_mld5.0_nc6_cvd100_minj40_maxj80_ns10000_f.txt",
    8: "../set-generation/colors_mcd18.0_mld4.2_nc8_cvd100_minj40_maxj82_ns10000_f.txt",
    10: "../set-generation/colors_mcd16.0_mld3.6_nc10_cvd100_minj40_maxj84_ns10000_f.txt",
}

In [11]:
with open("max-min-dist-sets.json") as infile:
    max_min_dist_sets = json.load(infile)
max_min_dist_sets = {int(i): np.array(max_min_dist_sets[i]) for i in max_min_dist_sets}
for nc in ALL_NUM_COLORS:
    rgb = [
        (int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in max_min_dist_sets[nc]
    ]
    jab = [colorspacious.cspace_convert(i, "sRGB255", "CAM02-UCS") for i in rgb]
    hcl = np.array(
        [[np.arctan2(i[2], i[1]), np.sqrt(i[1] ** 2 + i[2] ** 2), i[0]] for i in jab]
    )
    max_min_dist_sets[nc] = max_min_dist_sets[nc][np.lexsort(hcl[:, ::-1].T)]

In [12]:
# Load color data and sort by hue, chroma, lightness
colors_rgb = {}
for num_colors in ALL_NUM_COLORS:
    with open(COLOR_FILE[num_colors]) as csv_file:
        # Skip header rows
        csv_file.readline()
        csv_file.readline()
        csv_file.readline()
        csv_reader = csv.reader(csv_file, delimiter=" ")
        colors_rgb[num_colors] = []
        for row in csv_reader:
            row = [i.strip() for i in row]
            rgb = [(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in row]
            jab = [colorspacious.cspace_convert(i, "sRGB255", "CAM02-UCS") for i in rgb]
            hcl = np.array(
                [
                    [np.arctan2(i[2], i[1]), np.sqrt(i[1] ** 2 + i[2] ** 2), i[0]]
                    for i in jab
                ]
            )
            new_row = np.array(row)[np.lexsort(hcl[:, ::-1].T)]
            colors_rgb[num_colors].append(new_row)
    colors_rgb[num_colors] = np.array(colors_rgb[num_colors])

In [13]:
npz = np.load("../aesthetic-models/set-scores.npz")
scores = {}
for i in ALL_NUM_COLORS:
    mean = npz[f"mean{i:02d}"]
    saliency = npz[f"saliency{i:02d}"]
    scores[i] = mean * saliency

In [14]:
argsorts = {nc: np.argsort(scores[nc]) for nc in ALL_NUM_COLORS}
print(
    r"""\begin{tabular}{@{}rccc@{}}
\toprule
Rank & Six Colors & Eight Colors & Ten Colors \\
\midrule"""
)
lines = []
for i in range(9990, 10000):
    line = f"{10000 - i} & "
    for nc in ALL_NUM_COLORS:
        for c in colors_rgb[nc][argsorts[nc][i]]:
            line += r"\textcolor[HTML]{" + c + r"}{$\blacksquare$}"
        line += r" & "
    lines.append(line[:-2] + r"\\")
for l in reversed(lines):
    print(l)
print(r"\midrule")
lines = []
for i in range(10):
    line = f"{10000 - i} & "
    for nc in ALL_NUM_COLORS:
        for c in colors_rgb[nc][argsorts[nc][i]]:
            line += r"\textcolor[HTML]{" + c + r"}{$\blacksquare$}"
        line += r" & "
    lines.append(line[:-2] + r"\\")
for l in reversed(lines):
    print(l)

print(r"\midrule")
line = r"$\max\min(\Delta E_\text{cvd})$ & "
for nc in ALL_NUM_COLORS:
    for c in max_min_dist_sets[nc]:
        line += r"\textcolor[HTML]{" + c + r"}{$\blacksquare$}"
    line += r" & "
print(line[:-2] + r"\\")

print(
    r"""\bottomrule
\end{tabular}"""
)

\begin{tabular}{@{}rccc@{}}
\toprule
Rank & Six Colors & Eight Colors & Ten Colors \\
\midrule
1 & \textcolor[HTML]{5790fc}{$\blacksquare$}\textcolor[HTML]{9c9ca1}{$\blacksquare$}\textcolor[HTML]{7a21dd}{$\blacksquare$}\textcolor[HTML]{964a8b}{$\blacksquare$}\textcolor[HTML]{e42536}{$\blacksquare$}\textcolor[HTML]{f89c20}{$\blacksquare$} & \textcolor[HTML]{86c8dd}{$\blacksquare$}\textcolor[HTML]{578dff}{$\blacksquare$}\textcolor[HTML]{1845fb}{$\blacksquare$}\textcolor[HTML]{656364}{$\blacksquare$}\textcolor[HTML]{c849a9}{$\blacksquare$}\textcolor[HTML]{c91f16}{$\blacksquare$}\textcolor[HTML]{ff5e02}{$\blacksquare$}\textcolor[HTML]{adad7d}{$\blacksquare$} & \textcolor[HTML]{94a4a2}{$\blacksquare$}\textcolor[HTML]{92dadd}{$\blacksquare$}\textcolor[HTML]{3f90da}{$\blacksquare$}\textcolor[HTML]{717581}{$\blacksquare$}\textcolor[HTML]{832db6}{$\blacksquare$}\textcolor[HTML]{bd1f01}{$\blacksquare$}\textcolor[HTML]{a96b59}{$\blacksquare$}\textcolor[HTML]{e76300}{$\blacksquare$}\textcolor[HTML